# 1. Load the libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import cv2
import glob
import numpy as np
import pandas as pd  
from PIL import Image
from skimage import measure
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patches as patches
import tensorflow as tf
from skimage.measure import label, regionprops, regionprops_table

# 2. ResNet50

## 2.1 Load the model

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D, MaxPooling2D, AveragePooling2D
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import load_model


#load pre-trained model
model = ResNet50(include_top=False, weights="imagenet", input_shape=(50, 50, 3))
model.summary()
'''

#load trained model with binary classification
trained_model = load_model("/home/gauss/Desktop/Training_KB/KB_DMSO_DRUG_3/Trained_Model")
trained_model.summary()
'''

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50, 50, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 56, 56, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 25, 25, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 25, 25, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 13, 13, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 13, 13, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 13, 13, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 13, 13, 256)  1024       ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 7, 7, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 7, 7, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 7, 7, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 4, 4, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 4, 4, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 4, 4, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 4, 4, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 4, 4, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 2, 2, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 2, 2, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 2, 2, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

'\n\n#load trained model with binary classification\ntrained_model = load_model("/home/gauss/Desktop/Training_KB/KB_DMSO_DRUG_3/Trained_Model")\ntrained_model.summary()\n'

## 2.2 Assign Feature extraction layer

In [3]:

#feature extraction from pre-trained model
layer = model.get_layer(name="conv5_block3_2_conv").output
output = GlobalAveragePooling2D()(layer)
# define new model
feature_extraction_model = Model(inputs=model.inputs, outputs=output)
# summarize
feature_extraction_model.summary()
'''


# My `trained_model` model
dense_layer_output = trained_model.get_layer(name="dense").output
# Create a feature extraction model
feature_extraction_model = tf.keras.Model(inputs=trained_model.input, outputs=dense_layer_output)
feature_extraction_model.summary()
'''

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50, 50, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 56, 56, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 25, 25, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 25, 25, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 13, 13, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 13, 13, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 13, 13, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 13, 13, 256)  1024       ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 7, 7, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 7, 7, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 7, 7, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 4, 4, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 4, 4, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 4, 4, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 4, 4, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 4, 4, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 2, 2, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 2, 2, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 2, 2, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

'\n\n\n# My `trained_model` model\ndense_layer_output = trained_model.get_layer(name="dense").output\n# Create a feature extraction model\nfeature_extraction_model = tf.keras.Model(inputs=trained_model.input, outputs=dense_layer_output)\nfeature_extraction_model.summary()\n'

In [4]:
#define the file paths for the images and corresponding masks
image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N8863__2020-04-08T11_42_16-Measurement 1'
mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N8863__2020-04-08T11_42_16-Measurement 1'

#use the glob library to generate lists of image and mask filenames
images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])


ch1 = []
ch2 = []
ch4 = []

for i in range(0, 1152, 3):
    
    CH1_path = os.path.join(image_path, images[i])
    CH2_path = os.path.join(image_path, images[i+1])
    CH4_path = os.path.join(image_path, images[i+2])

    
    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(CH1_path)
    ch2_img = Image.open(CH2_path)
    ch4_img = Image.open(CH4_path)

    ch1.append(np.array(ch1_img))
    ch2.append(np.array(ch2_img))
    ch4.append(np.array(ch4_img))

ch1_max = np.max(ch1)
ch2_max = np.max(ch2)
ch4_max = np.max(ch4)
print("Maximum intensity for channel 1:", ch1_max)
print("Maximum intensity for channel 2:", ch2_max)
print("Maximum intensity for channel 4:", ch4_max)

ch1_q099 =  np.quantile(ch1, 0.99)
ch2_q099 =  np.quantile(ch2, 0.99)
ch4_q099 =  np.quantile(ch4, 0.99)
print("Quantile_099 channel 1:", ch1_q099)
print("Quantile_099 channel 2:", ch2_q099)
print("Quantile_099 channel 4:", ch4_q099)

Maximum intensity for channel 1: 65314
Maximum intensity for channel 2: 65322
Maximum intensity for channel 4: 62641
Quantile_099 channel 1: 1848.0
Quantile_099 channel 2: 5207.0
Quantile_099 channel 4: 2457.0


## 2.3 Read images as tensors and extract features

In [ ]:
#define the file paths for the images and corresponding masks
image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N8863__2020-04-08T11_42_16-Measurement 1'
mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N8863__2020-04-08T11_42_16-Measurement 1'

#use the glob library to generate lists of image and mask filenames
images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])

# "mean_feature_list" and "j" variables are initialized to empty list and 0, respectively to store the features for each image and to iterate over the list of masks.
mean_fetaure_list =[]
image_list = []
j=0

#loop with step size of 3 to iterate over a range of indices, pulling the filenames for three channels of each image.
for i in range(0, 1152, 3):
    
    CH1_path = os.path.join(image_path, images[i])
    CH2_path = os.path.join(image_path, images[i+1])
    CH4_path = os.path.join(image_path, images[i+2])
    
    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(CH1_path)
    ch2_img = Image.open(CH2_path)
    ch4_img = Image.open(CH4_path)

    ch1_img_arr = np.array(ch1_img)
    ch2_img_arr = np.array(ch2_img)
    ch4_img_arr = np.array(ch4_img)
    

    #use cellpose mask to extract individual cell regions,to generate 50x50 pixel images.
    cellpose_path = os.path.join(mask_path, cellpose_mask[j])

    cellpose_img = Image.open(cellpose_path)
    cellpose_img_array = np.array(cellpose_img)
    labels = label(cellpose_img_array)
    regions = measure.regionprops(labels)

    j=j+1
    single_cell_bbox = []

    for props in regions:
        y0, x0 = props.centroid
        y = int(round(y0))
        x = int(round(x0))
        box_ch1 = ch1_img_arr[x-25:x+25,y-25:y+25]
        box_ch2 = ch2_img_arr[x-25:x+25,y-25:y+25]
        box_ch4 = ch4_img_arr[x-25:x+25,y-25:y+25]

        if (box_ch1.shape == (50,50)):
            coordinate = x, y
            
            box_ch1 = (box_ch1.astype('float32')/ch1_q099)*255
            box_ch2 = (box_ch2.astype('float32')/ch2_q099)*255
            box_ch4 = (box_ch4.astype('float32')/ch4_q099)*255
            
            singlecell = np.stack((box_ch1, box_ch2, box_ch4), axis=-1)
            reshaped_singlecell = singlecell.reshape((1,50,50,3))
            single_cell_bbox.append(reshaped_singlecell)

    single_cells = np.array(single_cell_bbox)

    if len(single_cells) != 0:
        
        print(images[i])
    
        #convert single cell objects into a tensor using tf.data.Dataset.from_tensor_slices.
        single_cell_tensor = tf.data.Dataset.from_tensor_slices(single_cells)

        def extract_features(image):
            features = feature_extraction_model(image)
            return features

        #A map operation is used to apply a feature extraction function, extract_features, to each cell image 
        #using a pre-trained model. This creates a feature_dataset of extracted features.
        feature_dataset = single_cell_tensor.map(extract_features)

        #features are summed across all cells, divided by the number of cells, and added to the mean_feature_list.
        sum_feature_dataset = tf.zeros((1,512))

        for f in feature_dataset:
            sum_feature_dataset += f

        mean_feature = sum_feature_dataset/len(feature_dataset)

        mean_fetaure_list.append(mean_feature)
        image_list.append(images[i])

r01c01ch1.tiff
r01c02ch1.tiff
r01c03ch1.tiff
r01c04ch1.tiff
r01c05ch1.tiff


In [ ]:
#Convert mean feature tensor array to numpy array
feature_list = [tensor.numpy() for tensor in mean_fetaure_list]

In [ ]:
len(feature_list)

In [ ]:
# Create a 2D numpy array from the feature list
feature_array = np.vstack(feature_list)

# Create a DataFrame with column names
df = pd.DataFrame(feature_array, columns=['feature{}'.format(i+1) for i in range(512)])

#add col to the beginning of the dataframe
df.insert(0, 'Image', range(1,len(feature_list)+1))
df

In [ ]:
#save dataframe as excel 
df.to_excel("/home/gauss/Desktop/Pretrained Features/MHB/N8863.xlsx")


In [13]:
len(image_list)

381

In [17]:
image_list[300:384]

['r13c15ch1.tiff',
 'r13c16ch1.tiff',
 'r13c17ch1.tiff',
 'r13c18ch1.tiff',
 'r13c19ch1.tiff',
 'r13c20ch1.tiff',
 'r13c21ch1.tiff',
 'r13c22ch1.tiff',
 'r13c23ch1.tiff',
 'r13c24ch1.tiff',
 'r14c01ch1.tiff',
 'r14c02ch1.tiff',
 'r14c03ch1.tiff',
 'r14c04ch1.tiff',
 'r14c05ch1.tiff',
 'r14c06ch1.tiff',
 'r14c07ch1.tiff',
 'r14c08ch1.tiff',
 'r14c09ch1.tiff',
 'r14c10ch1.tiff',
 'r14c11ch1.tiff',
 'r14c12ch1.tiff',
 'r14c13ch1.tiff',
 'r14c14ch1.tiff',
 'r14c15ch1.tiff',
 'r14c16ch1.tiff',
 'r14c17ch1.tiff',
 'r14c18ch1.tiff',
 'r14c19ch1.tiff',
 'r14c20ch1.tiff',
 'r14c21ch1.tiff',
 'r14c22ch1.tiff',
 'r14c23ch1.tiff',
 'r14c24ch1.tiff',
 'r15c01ch1.tiff',
 'r15c02ch1.tiff',
 'r15c03ch1.tiff',
 'r15c04ch1.tiff',
 'r15c05ch1.tiff',
 'r15c06ch1.tiff',
 'r15c07ch1.tiff',
 'r15c08ch1.tiff',
 'r15c09ch1.tiff',
 'r15c10ch1.tiff',
 'r15c11ch1.tiff',
 'r15c12ch1.tiff',
 'r15c13ch1.tiff',
 'r15c14ch1.tiff',
 'r15c15ch1.tiff',
 'r15c16ch1.tiff',
 'r15c18ch1.tiff',
 'r15c19ch1.tiff',
 'r15c20ch1.